<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/%EB%B9%85%EC%9D%91%EB%B3%B4_case6_AE_IF_LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:

data = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type6.csv')#.to_numpy()


X = data[data['type'] == 'Normal']
y = X['type']
y = y.map({'Normal': 0, 'Attack': 1}).to_numpy()
X = X.drop(columns =['type'])
X['messageID'] = X['messageID'].astype('category')
X = X.drop(columns =['isCan0','isService','serviceNODEID'])
X = X.drop(columns = ['data[0]','data[7]'])
print(X.shape)


(160233, 12)


In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]
encoding_dim = 6

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Autoencoder 학습
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/30
4006/4006 [==============================] - 12s 2ms/step - loss: 0.6725 - val_loss: 0.6452
Epoch 2/30
4006/4006 [==============================] - 10s 2ms/step - loss: 0.5913 - val_loss: 0.5821
Epoch 3/30
4006/4006 [==============================] - 10s 2ms/step - loss: 0.5853 - val_loss: 0.5670
Epoch 4/30
4006/4006 [==============================] - 9s 2ms/step - loss: 0.5828 - val_loss: 0.5493
Epoch 5/30
4006/4006 [==============================] - 10s 2ms/step - loss: 0.5812 - val_loss: 0.5478
Epoch 6/30
4006/4006 [==============================] - 9s 2ms/step - loss: 0.5807 - val_loss: 0.5379
Epoch 7/30
4006/4006 [==============================] - 10s 2ms/step - loss: 0.5798 - val_loss: 0.5360
Epoch 8/30
4006/4006 [==============================] - 9s 2ms/step - loss: 0.5795 - val_loss: 0.5356
Epoch 9/30
4006/4006 [==============================] - 9s 2ms/step - loss: 0.5793 - val_loss: 0.5351
Epoch 10/30
4006/4006 [==============================] - 10s 2ms/step - loss:

In [4]:
# 데이터를 인코딩
X_encoded = autoencoder.predict(X_scaled)

5008/5008 [==============================] - 6s 1ms/step


In [5]:
#data.loc[data['type']=='Normal', 'type'] = 0
#data.loc[data['type']=='Attack', 'type'] = 1

fullDataX1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type6.csv')#.to_numpy()

#X['messageID'] = data['messageID']
fX1 = fullDataX1.drop(columns =['type'])
fX1['messageID'] = fX1['messageID'].astype('category')
fX1 = fX1.drop(columns =['isCan0','isService','serviceNODEID'])
fX1 = fX1.drop(columns = ['data[0]','data[7]'])
print(fX1.shape)
fX1_scaled = scaler.fit_transform(fX1)
fX1_encoded = autoencoder.predict(fX1_scaled)

(241321, 12)
7542/7542 [==============================] - 9s 1ms/step


In [6]:
if_model = IsolationForest(contamination=0.1)  # contamination 값 조정 가능
if_model.fit(X_encoded)

# 이상치 예측
if_predictions = if_model.predict(fX1_encoded)

In [ ]:
fy = fullDataX1['type']
fy = fy.map({'Normal': 1, 'Attack': 0}).to_numpy()
if_predictions = np.where(if_predictions == -1, 0, if_predictions)
print(np.unique(if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy, if_predictions)
class_report = classification_report(fy, if_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([ 80392, 117087]))
Confusion Matrix:
[[61189 34689]
 [19203 82398]]

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.64      0.69     95878
           1       0.70      0.81      0.75    101601

    accuracy                           0.73    197479
   macro avg       0.73      0.72      0.72    197479
weighted avg       0.73      0.73      0.72    197479



In [7]:
# 모델 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case6_if_model.pkl', 'wb') as file:
    pickle.dump(if_model, file)

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case6_if_model.pkl', 'rb') as file:
    loaded_if_model = pickle.load(file)

# 불러온 모델로 예측
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 1, 'Attack': 0}).to_numpy()
loaded_if_predictions = loaded_if_model.predict(fX1_encoded)
loaded_if_predictions = np.where(loaded_if_predictions == -1, 0, loaded_if_predictions)
print(np.unique(loaded_if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, loaded_if_predictions)
class_report = classification_report(fy1, loaded_if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([ 93537, 103942]))
Confusion Matrix:
[[65180 30698]
 [28357 73244]]

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.68      0.69     95878
           1       0.70      0.72      0.71    101601

    accuracy                           0.70    197479
   macro avg       0.70      0.70      0.70    197479
weighted avg       0.70      0.70      0.70    197479



In [ ]:
# Local Outlier Factor 모델 정의 및 학습
lof_model = LocalOutlierFactor(n_neighbors=500, contamination=0.5, novelty=True)  # n_neighbors 및 contamination 값 조정 가능
lof_model.fit(X_encoded)
lof_predictions = lof_model.predict(fX1_encoded)
# 이상치라면 -1, 정상치라면 1로 표시됨


In [ ]:
fy = fullDataX1['type']
fy = fy.map({'Normal': 1, 'Attack': 0}).to_numpy()
lof_predictions = np.where(lof_predictions == -1, 0, lof_predictions)
print(np.unique(lof_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy, lof_predictions)
class_report = classification_report(fy, lof_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([191634,   5845]))
Confusion Matrix:
[[94173  1705]
 [97461  4140]]

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.98      0.66     95878
           1       0.71      0.04      0.08    101601

    accuracy                           0.50    197479
   macro avg       0.60      0.51      0.37    197479
weighted avg       0.60      0.50      0.36    197479



In [ ]:
# 모델 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_lof_model.pkl', 'wb') as file:
    pickle.dump(lof_model, file)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_lof_model.pkl', 'rb') as file:
    loaded_lof_model = pickle.load(file)

# 불러온 모델로 예측
loaded_lof_predictions = loaded_lof_model.predict(fX1_encoded)
print(np.unique(loaded_lof_predictions, return_counts=True))